# Classification with CNN : Cats vs Dogs

## Step 0. Library Import

In [2]:
import tensorflow as tf

import tensorflow_datasets as tfds

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

## Step 2. Dataset Load

In [3]:
dataset_name = 'cats_vs_dogs'
train_set = tfds.load(name=dataset_name, split='train[:80%]')
valid_set = tfds.load(name=dataset_name, split='train[80%:]')

## Step 3. Data Preprocessing

In [4]:
def preprocess(data):
    x = data['image'] / 255
    x = tf.image.resize(x, size=(224, 224))
    
    y = data['label']

    return x, y

In [5]:
batch_size = 128
train_data = train_set.map(preprocess).batch(batch_size)
valid_data = valid_set.map(preprocess).batch(batch_size)

## Step 4. Modeling

In [6]:
model = Sequential([
    Conv2D(64, (3, 3), input_shape=(224, 224, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax'),
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       1

## Step 5. Model Compile

In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

## Step 6. Model Checkpoint

In [9]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

## Step 7. Model Fit

In [10]:
epochs=12

In [33]:
model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    callbacks=[cp]
)

Epoch 1/12
 1/37 [..............................] - ETA: 0s - loss: 0.6935 - acc: 0.5000